In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np

PART I - Transforming an image from color to grayscale

In [ ]:
filename = '../dataset/tiger_320x240.ppm'
img = cv2.imread(filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

r_img = img[:, :, 0]
g_img = img[:, :, 1]
b_img = img[:, :, 2]

gamma = 1.400  # a parameter
r_const, g_const, b_const = 0.2126, 0.7152, 0.0722  # weights for the RGB components respectively
grayscale_image = r_const * r_img ** gamma + g_const * g_img ** gamma + b_const * b_img ** gamma
print(f"grayscale[0,0]: {grayscale_image[0,0]}")

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(img)
axes[0].set_title("Image originale")
axes[0].axis('on')

display_img = grayscale_image.copy()
display_img = ((display_img - display_img.min()) / (display_img.max() - display_img.min()) * 255).astype(np.uint8)
axes[1].imshow(display_img, cmap=plt.get_cmap('gray'))
axes[1].set_title("Image en niveaux de gris")
axes[1].axis('on')
print(f"grayscale_display[0,0]: {display_img[0,0]}")

plt.tight_layout()
plt.show()

PART II - Applying the Sobel operator

In [ ]:
"""
The kernels Gx and Gy can be thought of as a differential operation in the "input_image" array in the directions x and y 
respectively. These kernels are represented by the following matrices:
      _               _                   _                _
     |                 |                 |                  |
     | 1.0   0.0  -1.0 |                 |  1.0   2.0   1.0 |
Gx = | 2.0   0.0  -2.0 |    and     Gy = |  0.0   0.0   0.0 |
     | 1.0   0.0  -1.0 |                 | -1.0  -2.0  -1.0 |
     |_               _|                 |_                _|
"""

import numpy as np
import matplotlib.pyplot as plt

# Here we define the matrices associated with the Sobel filter
Gx = np.array([[1.0, 0.0, -1.0], [2.0, 0.0, -2.0], [1.0, 0.0, -1.0]])
Gy = np.array([[1.0, 2.0, 1.0], [0.0, 0.0, 0.0], [-1.0, -2.0, -1.0]])
[rows, columns] = np.shape(grayscale_image)  # we need to know the shape of the input grayscale image

# Initialize output image
sobel_filtered_image = np.zeros(shape=(rows, columns))

# Now we "sweep" the image in both x and y directions and compute the output
for i in range(rows - 2):
    for j in range(columns - 2):
        gx = np.sum(np.multiply(Gx, grayscale_image[i:i + 3, j:j + 3]))  # x direction
        gy = np.sum(np.multiply(Gy, grayscale_image[i:i + 3, j:j + 3]))  # y direction
        
        # Hardware-friendly approximation: |Gx| + |Gy| (no square root)
        sobel_filtered_image[i + 1, j + 1] = np.abs(gx) + np.abs(gy)

# Normalize for display (0-255 range)
if sobel_filtered_image.max() > sobel_filtered_image.min():
    display_img = ((sobel_filtered_image - sobel_filtered_image.min()) / 
                   (sobel_filtered_image.max() - sobel_filtered_image.min()) * 255).astype(np.uint8)
else:
    display_img = np.zeros_like(sobel_filtered_image, dtype=np.uint8)

# Display results
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
ax1, ax2 = axes[0], axes[1]

# Original image
ax1.imshow(img)
ax1.set_title("Image originale")
ax1.axis('on')

# Sobel filtered image with hardware-friendly approximation
ax2.imshow(display_img, cmap='gray')
ax2.set_title("Sobel\n|Gx| + |Gy| approximation")
ax2.axis('on')

plt.tight_layout()
plt.show()